In [1]:
import sys
sys.path.append('../')

In [3]:
import numpy as np
import pandas as pd

# from experiment import data_generation, behavior_and_evaluation_policy, true_value
from kernel_regression import KernelRegression
# from cs_ope_estimator import ipw, dm, dr, dml 
from ucimlrepo import fetch_ucirepo 

In [4]:
from statsmodels.nonparametric.kernel_regression import KernelReg

In [58]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

In [195]:
import numpy as np
import argparse
import sys
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_svmlight_file
import sys
sys.path.append('../')

from cs_opl import op_learning
from experiment_learning import *

False


RuntimeError: No CUDA GPUs are available

In [59]:
def convert_labels_to_numbers(data):
    if isinstance(data, pd.Series):  # Kiểm tra nếu là Pandas Series
        # Kiểm tra nếu dữ liệu là kiểu số (int hoặc float), thì giữ nguyên
        if pd.api.types.is_numeric_dtype(data):
            return data
        else:
            # Nếu là kiểu categorical (text), sử dụng LabelEncoder để chuyển thành số
            encoder = LabelEncoder()
            return encoder.fit_transform(data)

In [13]:
N=300
ration=0.7
data_name='heart_disease'

In [173]:
label_encoder = LabelEncoder()

In [ ]:
heart_disease = fetch_ucirepo(id=45) 
  
# data (as pandas dataframes) 
X = heart_disease.data.features 
Y = heart_disease.data.targets 
name=Y.columns.tolist()
Y['new_label']=Y[name[0]].astype(str) + '_' + Y[name[1]].astype(str)
Y= label_encoder.fit_transform(Y['new_label'])
X = X.interpolate().values
X = X/X.max(axis=0)
Y=np.squeeze(Y)

Y = np.array(Y, np.int64)
N_train = int(N*ration)
N_test= N - N_train

perm = np.random.permutation(len(X))

X, Y = X[perm[:N]], Y[perm[:N]]

if data_name == 'satimage.scale':
    Y = Y - 1
elif data_name == 'vehicle.scale':
    Y = Y - 1
unique_labels = np.sort(np.unique(Y))
label_mapping = {label: idx for idx, label in enumerate(unique_labels)}

Y=np.array([label_mapping[label] for label in Y])
classes = np.unique(Y)
Y_matrix = np.zeros(shape=(N, len(classes)))
print(Y_matrix.shape)
for i in range(N):
    Y_matrix[i, Y[i]] = 1

prob = 1/(1+np.exp(-(X[:,0]+X[:,1]+X[:,2]+X[:,3]+X[:,4]+0.1*np.random.normal(size=len(X)))))
rand = np.random.uniform(size=len(X))

prob_base = prob

for C in range(100000):
    C /= 1000
    eval_samplesize = np.sum(prob*C > rand)
    if eval_samplesize > N_train:
        break
    prob_base = prob*C

train_test_split = prob_base > rand

if np.sum(train_test_split) != N_train:
    N_train = np.sum(train_test_split)
    N_test = N - N_train



(300, 13)


/tmp/ipykernel_102084/3537203937.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Y['new_label']=Y[name[0]].astype(str) + '_' + Y[name[1]].astype(str)


In [41]:
Y[0]

array([0])

In [ ]:
X_train, X_test = X[train_test_split], X[~train_test_split]

    Y_matrix_train, Y_matrix_test = Y_matrix[train_test_split], Y_matrix[~train_test_split]

    for idx_alpha in  range(len(alphas)):    
        alpha = alphas[idx_alpha]

        pi_behavior, pi_evaluation  = behavior_and_evaluation_policy(X, Y, train_test_split, classes, alpha=alpha)

        pi_behavior_train = pi_behavior[train_test_split]
        pi_evaluation_train, pi_evaluation_test = pi_evaluation[train_test_split], pi_evaluation[~train_test_split]

        tau = true_value(Y_matrix_test, pi_evaluation_test, N_test)
        tau_list[trial] = tau

        perm = np.random.permutation(N_train)
        X_seq_train, Y_matrix_seq_train, pi_behavior_seq_train, pi_evaluation_seq_train = X_train[perm], Y_matrix_train[perm], pi_behavior_train[perm], pi_evaluation_train[perm]

        Y_historical_matrix = np.zeros(shape=(N_train, len(classes)))
        A_historical_matrix = np.zeros(shape=(N_train, len(classes)))
        Action=np.zeros(shape=(N_train),dtype=np.int16)

        for i in range(N_train):
            a = np.random.choice(classes, p=pi_behavior_seq_train[i])
            Y_historical_matrix[i, a] = Y_matrix_seq_train[i, a]
            A_historical_matrix[i, a] = 1
            Action[i]=a

        # dataset = (contexts, actions, rewards, test_contexts, mean_test_rewards)

        estimators = op_learning(X_seq_train, A_historical_matrix, Action, Y_historical_matrix, X_test, Y_matrix_test,classes,args,lcb=True)



In [ ]:
X, Y, Y_matrix, train_test_split, x_prob, classes, N, N_train, N_test = data_generation('satimage', sample_size,ratio)

TypeError: data_generation() missing 2 required positional arguments: 'N' and 'ration'

In [136]:
Y = Y[:10]
X = X[:10]

In [137]:
model = KernelReg(Y, X, var_type='c'*X.shape[1], reg_type='lc')

In [142]:
X[0]  = 1000000

In [143]:
d = model.fit(X[:1000])

/Users/mkato/opt/anaconda3/lib/python3.7/site-packages/statsmodels/nonparametric/kernel_regression.py:224: RuntimeWarning: invalid value encountered in true_divide
  G = G_numer / G_denom
/Users/mkato/opt/anaconda3/lib/python3.7/site-packages/statsmodels/nonparametric/kernel_regression.py:236: RuntimeWarning: invalid value encountered in true_divide
  B_x = d_mx / f_x - G * d_fx / f_x


In [144]:
d

(array([           nan, 2.00000000e+00, 1.00000000e+00, 4.00000000e+00,
        1.00000000e+00, 6.00000000e+00, 4.00000000e+00, 1.13051313e-15,
        5.00000000e+00, 1.08135360e-17, 3.99999114e+00, 7.99999979e+00,
        1.00000000e+00, 3.99957528e+00, 3.99916545e+00, 1.00000000e+00,
        9.99999796e-01, 9.99828921e-01, 2.00000000e+00, 1.99999997e+00,
        5.00000000e+00, 3.99999902e+00, 5.00000000e+00, 2.92660586e-01,
        3.90704412e+00, 4.00000000e+00, 8.38081644e-18, 1.00000000e+00,
        3.61579444e+00, 5.92738751e+00, 3.26006364e-02, 5.00000000e+00,
        1.99981557e+00, 3.28135976e-18, 3.99999271e+00, 1.00000000e+00,
        5.00000000e+00, 4.00000000e+00, 6.00000000e+00, 2.00000000e+00,
        1.00001128e+00, 5.00000000e+00, 1.38543591e+00, 1.00000000e+00,
        1.00000000e+00, 5.99995998e+00, 9.99999945e-01, 2.00000000e+00,
        5.00000000e+00, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
        2.69329892e-15, 1.00000000e+00, 4.00000000e+00, 5.000000

In [88]:
from sklearn.kernel_ridge import KernelRidge

In [89]:
clf = KernelRidge(alpha=0.01)
clf.fit(X, Y)

KernelRidge(alpha=0.01, coef0=1, degree=3, gamma=None, kernel='linear',
            kernel_params=None)

In [106]:
from sklearn import linear_model

In [115]:
clf = linear_model.LogisticRegression(penalty='l1', C=0.1, solver='saga', multi_class='multi_class')

In [116]:
Y

array([8, 2, 1, ..., 5, 1, 7])

In [117]:
a, b = np.where(Y_matrix == 1)

In [118]:
b

array([8, 2, 1, ..., 5, 1, 7])

In [120]:
Y

array([8, 2, 1, ..., 5, 1, 7])

In [25]:
model = KernelReg([1,100,1], X, var_type='c')

ValueError: cannot reshape array of size 159660 into shape (4435,1)

In [28]:
import statsmodels.api as sm

In [33]:
'3'*3

'333'

In [35]:
'c'*X.shape[1]

'cccccccccccccccccccccccccccccccccccc'

In [36]:
dens_u = sm.nonparametric.KDEMultivariate(data=X, var_type='c'*dim, bw='normal_reference')

In [37]:
dens_u.pdf(X[0])

array(0.56532044)

In [69]:
bw = dens_u.bw

In [102]:
Y = Y[:1000]
X = X[:50]

In [ ]:
model = KernelReg(Y, X, var_type='c'*X.shape[1], reg_type='lc')

In [103]:
dens_u = sm.nonparametric.KDEMultivariate(data=X, var_type='c'*X.shape[1], bw='normal_reference')

In [104]:
model = KernelReg(Y, X, var_type='c'*X.shape[1], reg_type='lc')

In [98]:
model.bw

array([0.19174831, 0.28450028, 0.27009914, 0.15901443, 0.30847048,
       0.28977343, 0.17193598, 0.03891559, 0.20149673, 0.31748389,
       0.28018324, 0.28895554, 0.15374873, 0.21203015, 0.57902868,
       0.05410153])

In [85]:
v, x = model.fit(X)

In [87]:
x.shape

(3000, 16)

In [60]:
X.size

119904

In [55]:
model.fit(X[-16:])

(array([4.73930242, 4.83120944, 4.76830902, 4.05162109, 4.54722321,
        4.90765989, 4.45411434, 4.89109568, 4.71458883, 4.78442258,
        4.7987667 , 4.75148552, 4.46570638, 4.97899378, 4.98634629,
        4.81090934]),
 array([[-2.42074379e+15, -2.42074379e+15, -2.42074379e+15,
         -2.42074379e+15, -2.42074379e+15, -2.42074379e+15,
         -2.42074379e+15, -2.42074379e+15, -2.42074379e+15,
         -2.42074379e+15, -2.42074379e+15, -2.42074379e+15,
         -2.42074379e+15, -2.42074379e+15, -2.42074379e+15,
         -2.42074379e+15, -2.42074379e+15, -2.42074379e+15,
         -2.42074379e+15, -2.42074379e+15, -2.42074379e+15,
         -2.42074379e+15, -2.42074379e+15, -2.42074379e+15,
         -2.42074379e+15, -2.42074379e+15, -2.42074379e+15,
         -2.42074379e+15, -2.42074379e+15, -2.42074379e+15,
         -2.42074379e+15, -2.42074379e+15, -2.42074379e+15,
         -2.42074379e+15, -2.42074379e+15, -2.42074379e+15],
        [-1.80097159e+16, -1.80097159e+16, -1.8009715

In [26]:
X.shape

(4435, 36)

In [24]:
model.fit([1])

(array([1.]), array([[-99.]]))

In [14]:
Y

array([0, 0, 0, ..., 5, 5, 5])

In [4]:
num_trials = 100
alphas = [0.7, 0.4, 0.0]

tau_list = np.zeros(num_trials)
res_ipw3_list = np.zeros((num_trials, len(alphas)))
res_dm_list = np.zeros((num_trials, len(alphas)))
res_dml1_truew_list = np.zeros((num_trials, len(alphas)))
res_dml2_truew_list = np.zeros((num_trials, len(alphas)))

for trial in range(num_trials):
    X, Y, Y_matrix, train_test_split, x_prob, classes, N, N_train, N_test = data_generation(data_name)

    X_train, X_test = X[train_test_split], X[~train_test_split]
    Y_train, Y_test = Y[train_test_split], Y[~train_test_split]
    Y_matrix_train, Y_matrix_test = Y_matrix[train_test_split], Y_matrix[~train_test_split]

    pi_behavior, pi_evaluation  = behavior_and_evaluation_policy(X, Y, train_test_split, classes, alpha=0.7)

    pi_behavior_train, pi_behavior_test = pi_behavior[train_test_split], pi_behavior[~train_test_split]
    pi_evaluation_train, pi_evaluation_test = pi_evaluation[train_test_split], pi_evaluation[~train_test_split]

    tau = true_value(Y_matrix_test, pi_evaluation_test, N_test)

    for idx_alpha in  range(len(alphas)):    
        alpha = alphas[idx_alpha]
        pi_behavior, pi_evaluation  = behavior_and_evaluation_policy(X, Y, train_test_split, classes, alpha=alpha)

        perm = np.random.permutation(N_train)

        X_seq_train, Y_matrix_seq_train, pi_behavior_seq_train, pi_evaluation_seq_train = X_train[perm], Y_matrix_train[perm], pi_behavior_train[perm], pi_evaluation_train[perm]

        Y_historical_matrix = np.zeros(shape=(N_train, len(classes)))
        A_historical_matrix = np.zeros(shape=(N_train, len(classes)))

        for i in range(N_train):
            a = np.random.choice(classes, p=pi_behavior[i])
            Y_historical_matrix[i, a] = 1
            A_historical_matrix[i, a] = 1
            
        #IPW3 estimator
        res_ipw3 = ipw(Y_historical_matrix, X_seq_train, X_test, classes, pi_evaluation_train, A_hst=A_historical_matrix)
        #Direct method estimator
        res_dm = dm(Y_historical_matrix, X_seq_train, X_test, pi_evaluation_test, classes)
        #DML with L1
        res_dml1 =dml(Y_historical_matrix, A_historical_matrix, X_seq_train, X_test, pi_evaluation_seq_train, pi_evaluation_test, classes, pi_behavior=pi_behavior_seq_train, method='Lasso')
        #DML with L2
        res_dml2 =dml(Y_historical_matrix, A_historical_matrix, X_seq_train, X_test, pi_evaluation_seq_train, pi_evaluation_test, classes, pi_behavior=pi_behavior_seq_train, method='Ridge')

        print(res_ipw3)
        print(res_dm)
        print(res_dr1)
        print(res_dr2)
        print(res_dml1_truew)
        print(res_dml2_truew)
        
        res_ipw3_list[trial, idx_alpha] = res_ipw3
        res_dm_list[trial, idx_alpha] = res_dm
        res_dr1_list[trial, idx_alpha] = res_dr1
        res_dr2_list[trial, idx_alpha] = res_dr2
        res_dml1_truew_list[trial, idx_alpha] = res_dml1_truew
        res_dml2_truew_list[trial, idx_alpha] = res_dml2_truew
        
        np.savetxt("exp_results/res_ipw3.csv", res_ipw3_list, delimiter=",")
        np.savetxt("exp_results/res_dm.csv", res_dm_list, delimiter=",")
        np.savetxt("exp_results/res_dr1.csv", res_dr1_list, delimiter=",")
        np.savetxt("exp_results/res_dr2.csv", res_dr2_list, delimiter=",")
        np.savetxt("exp_results/res_dml1.csv", res_dml1_truew_list, delimiter=",")
        np.savetxt("exp_results/res_dml2.csv", res_dml2_truew_list, delimiter=",")
        
    tau_list[trial] = tau
        

[[0.2857443  0.1952922  0.26814298 0.07805563 0.05606263 0.11670226]
 [0.29536795 0.15428734 0.27987302 0.0802635  0.05640643 0.13380176]
 [0.37533892 0.1459109  0.18370155 0.12563121 0.06332002 0.10609741]
 ...
 [0.20679231 0.16011928 0.29419599 0.0838981  0.1119685  0.14302582]
 [0.31595816 0.13683452 0.25687274 0.10179922 0.11506038 0.07347497]
 [0.28837575 0.1668954  0.27622361 0.06731022 0.08418668 0.11700835]]
sigma:  0.001
lda:  0.01
sigma:  0.001
lda:  0.01
sigma:  0.001
lda:  0.01
sigma:  0.001
lda:  0.01
sigma:  0.001
lda:  0.01


/Users/mkato/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.08136092229213432, tolerance: 0.04414618372652263
  positive)
/Users/mkato/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.08136092229213432, tolerance: 0.04414618372652263
  positive)
/Users/mkato/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.09048368117350947, tolerance: 0.04430395631018423
  positive)
/Users/mkato/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You mi

sigma:  0.001
lda:  0.01
sigma:  0.001
lda:  0.01
sigma:  0.001
lda:  0.01
sigma:  0.001
lda:  0.01
sigma:  0.001
lda:  0.01
sigma:  0.001
lda:  0.01
sigma:  0.001
lda:  0.01
RuLSIF starting...
Searching for the optimal sigma and lambda...
sigma = 0.00100, lambda = 0.00100, score = 0.00000
sigma = 0.00100, lambda = 0.01000, score = 0.00000
sigma = 0.00100, lambda = 0.10000, score = 0.00000
sigma = 0.00100, lambda = 1.00000, score = 0.00000
sigma = 0.00100, lambda = 10.00000, score = -0.00000
sigma = 0.00100, lambda = 100.00000, score = 0.00000
sigma = 0.00100, lambda = 1000.00000, score = 0.00000
sigma = 0.00100, lambda = 10000.00000, score = -0.00000
sigma = 0.00100, lambda = 100000.00000, score = -0.00000
sigma = 0.00100, lambda = 1000000.00000, score = 0.00000
sigma = 0.00100, lambda = 10000000.00000, score = 0.00000
sigma = 0.00100, lambda = 100000000.00000, score = -0.00000
sigma = 0.00100, lambda = 1000000000.00000, score = 0.00000
sigma = 0.01000, lambda = 0.00100, score = -0.00

sigma = 100000000.00000, lambda = 0.00100, score = -0.50000
sigma = 100000000.00000, lambda = 0.01000, score = -0.50000
sigma = 100000000.00000, lambda = 0.10000, score = -0.50000
sigma = 100000000.00000, lambda = 1.00000, score = -0.49994
sigma = 100000000.00000, lambda = 10.00000, score = -0.49581
sigma = 100000000.00000, lambda = 100.00000, score = -0.37491
sigma = 100000000.00000, lambda = 1000.00000, score = -0.08677
sigma = 100000000.00000, lambda = 10000.00000, score = -0.00985
sigma = 100000000.00000, lambda = 100000.00000, score = -0.00100
sigma = 100000000.00000, lambda = 1000000.00000, score = -0.00010
sigma = 100000000.00000, lambda = 10000000.00000, score = -0.00001
sigma = 100000000.00000, lambda = 100000000.00000, score = -0.00000
sigma = 100000000.00000, lambda = 1000000000.00000, score = -0.00000
sigma = 1000000000.00000, lambda = 0.00100, score = -0.50000
sigma = 1000000000.00000, lambda = 0.01000, score = -0.50000
sigma = 1000000000.00000, lambda = 0.10000, score = -

sigma = 1000000.00000, lambda = 0.00100, score = -0.50000
sigma = 1000000.00000, lambda = 0.01000, score = -0.50000
sigma = 1000000.00000, lambda = 0.10000, score = -0.50000
sigma = 1000000.00000, lambda = 1.00000, score = -0.49994
sigma = 1000000.00000, lambda = 10.00000, score = -0.49581
sigma = 1000000.00000, lambda = 100.00000, score = -0.37491
sigma = 1000000.00000, lambda = 1000.00000, score = -0.08677
sigma = 1000000.00000, lambda = 10000.00000, score = -0.00985
sigma = 1000000.00000, lambda = 100000.00000, score = -0.00100
sigma = 1000000.00000, lambda = 1000000.00000, score = -0.00010
sigma = 1000000.00000, lambda = 10000000.00000, score = -0.00001
sigma = 1000000.00000, lambda = 100000000.00000, score = -0.00000
sigma = 1000000.00000, lambda = 1000000000.00000, score = -0.00000
sigma = 10000000.00000, lambda = 0.00100, score = -0.50000
sigma = 10000000.00000, lambda = 0.01000, score = -0.50000
sigma = 10000000.00000, lambda = 0.10000, score = -0.50000
sigma = 10000000.00000, 

...

sigma:  0.001
lda:  0.01
sigma:  0.001
lda:  0.01
sigma:  0.001
lda:  0.01
sigma:  0.001
lda:  0.01
sigma:  0.001
lda:  0.01
sigma:  0.001
lda:  0.01
sigma:  0.001
lda:  0.01
sigma:  0.001
lda:  0.01
sigma:  0.001
lda:  0.01
sigma:  0.001
lda:  0.01
sigma:  0.001
lda:  0.01
RuLSIF starting...
Searching for the optimal sigma and lambda...
sigma = 0.00100, lambda = 0.00100, score = 0.00000
sigma = 0.00100, lambda = 0.01000, score = 0.00000
sigma = 0.00100, lambda = 0.10000, score = 0.00000
sigma = 0.00100, lambda = 1.00000, score = 0.00000
sigma = 0.00100, lambda = 10.00000, score = -0.00000
sigma = 0.00100, lambda = 100.00000, score = 0.00000
sigma = 0.00100, lambda = 1000.00000, score = 0.00000
sigma = 0.00100, lambda = 10000.00000, score = -0.00000
sigma = 0.00100, lambda = 100000.00000, score = -0.00000
sigma = 0.00100, lambda = 1000000.00000, score = 0.00000
sigma = 0.00100, lambda = 10000000.00000, score = 0.00000
sigma = 0.00100, lambda = 100000000.00000, score = -0.00000
sigma = 

sigma = 100000000.00000, lambda = 0.00100, score = -0.50000
sigma = 100000000.00000, lambda = 0.01000, score = -0.50000
sigma = 100000000.00000, lambda = 0.10000, score = -0.50000
sigma = 100000000.00000, lambda = 1.00000, score = -0.49994
sigma = 100000000.00000, lambda = 10.00000, score = -0.49581
sigma = 100000000.00000, lambda = 100.00000, score = -0.37491
sigma = 100000000.00000, lambda = 1000.00000, score = -0.08677
sigma = 100000000.00000, lambda = 10000.00000, score = -0.00985
sigma = 100000000.00000, lambda = 100000.00000, score = -0.00100
sigma = 100000000.00000, lambda = 1000000.00000, score = -0.00010
sigma = 100000000.00000, lambda = 10000000.00000, score = -0.00001
sigma = 100000000.00000, lambda = 100000000.00000, score = -0.00000
sigma = 100000000.00000, lambda = 1000000000.00000, score = -0.00000
sigma = 1000000000.00000, lambda = 0.00100, score = -0.50000
sigma = 1000000000.00000, lambda = 0.01000, score = -0.50000
sigma = 1000000000.00000, lambda = 0.10000, score = -

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Users/mkato/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3326, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-4-afe327fa7d7e>", line 45, in <module>
    res_dr2 = dr(Y_historical_matrix, A_historical_matrix, X_seq_train, X_test, pi_evaluation_seq_train, pi_evaluation_test, classes, pi_behavior=pi_behavior_seq_train, method='Ridge')
  File "../cs_ope_estimator.py", line 79, in dr
    densratio_obj = densratio(X_evl, X_hst)
  File "/Users/mkato/opt/anaconda3/lib/python3.7/site-packages/densratio/core.py", line 66, in densratio
    result = RuLSIF(x, y, alpha, sigma_range, lambda_range, kernel_num, verbose)
  File "/Users/mkato/opt/anaconda3/lib/python3.7/site-packages/densratio/RuLSIF.py", line 61, in RuLSIF
    opt_params = search_sigma_and_lambda(x, y, alpha, centers, sigma_range, lambda_range, verbose)
  File "/Users/mkato/opt/anaconda3/lib/python3.7/site-packages

KeyboardInterrupt: 